#  RNN


In [3]:
import torch
import torch.nn as nn

# Download pretrained RNN Shakespeare model
checkpoint = torch.hub.load('spro/practical-nlp-code', 'shakespeare_rnn')
model = checkpoint['model']
stoi = checkpoint['stoi']
itos = checkpoint['itos']

def generate_rnn(model, seed="To be", length=200):
    model.eval()
    seq = torch.tensor([stoi[c] for c in seed]).unsqueeze(0)
    result = seed

    for _ in range(length):
        logits = model(seq)[:, -1, :]
        probs = torch.softmax(logits, dim=-1)
        next_id = torch.multinomial(probs, 1).item()
        result += itos[next_id]
        seq = torch.tensor([[next_id]])
    return result

print(generate_rnn(model))


HTTPError: HTTP Error 404: Not Found

In [5]:
import torch
import torch.nn as nn
import requests

# -------------------------------------------
# 1. LOAD SHAKESPEARE DATASET
# -------------------------------------------
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
text = requests.get(url).text

chars = sorted(list(set(text)))
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for ch,i in stoi.items()}

vocab_size = len(chars)

# -------------------------------------------
# 2. DEFINE A SIMPLE CHARACTER RNN MODEL
# -------------------------------------------
class CharRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size=256):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, vocab_size)
        self.rnn = nn.RNN(vocab_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h=None):
        x = self.embed(x)
        out, h = self.rnn(x, h)
        out = self.fc(out)
        return out, h

model = CharRNN(vocab_size)

# -------------------------------------------
# 3. TEXT GENERATION FUNCTION
# -------------------------------------------
def generate_text(model, seed="To be", length=200):
    model.eval()
    input_ids = torch.tensor([[stoi[c] for c in seed]])
    h = None
    result = seed

    for _ in range(length):
        logits, h = model(input_ids, h)
        probs = torch.softmax(logits[:, -1, :], dim=-1)
        next_id = torch.multinomial(probs, 1).item()
        result += itos[next_id]
        input_ids = torch.tensor([[next_id]])

    return result

# -------------------------------------------
# 4. TEST GENERATION (MODEL IS UNTRAINED)
# -------------------------------------------
print(generate_text(model))


To beUK;Yh3psO?lcEVmmXAWF-H lxoSruVcfBPvf3j;Y:VwilAEGy!qztouYmsnFKA3dQJSGgjGu;NKCk
lbKojs.
bifC.kdqBMI3tY
Wg,C cCziW-$,B$.dTAUnwG!v3fkp,
zBZzIxzpFbyBJRTXR:Yb-SZr,hq:IqZWJ!MoAwUzRmSCWyIX:$a!JifnRqgOges!JYR3


# LSTM

In [6]:
import torch
import torch.nn as nn

checkpoint = torch.hub.load('spro/practical-nlp-code', 'shakespeare_lstm')
model = checkpoint['model']
stoi = checkpoint['stoi']
itos = checkpoint['itos']

def generate_lstm(model, seed="My love", length=200):
    model.eval()
    seq = torch.tensor([stoi[c] for c in seed]).unsqueeze(0)
    result = seed

    for _ in range(length):
        logits = model(seq)[:, -1, :]
        probs = torch.softmax(logits, dim=-1)
        next_id = torch.multinomial(probs, 1).item()
        result += itos[next_id]
        seq = torch.tensor([[next_id]])
    return result

print(generate_lstm(model))


/Users/mac/anaconda3/lib/python3.11/site-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(


HTTPError: HTTP Error 404: Not Found

In [7]:
import torch
import torch.nn as nn
import requests

# -------------------------------------------
# 1. Load Shakespeare Dataset
# -------------------------------------------
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
text = requests.get(url).text

chars = sorted(list(set(text)))
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}
vocab_size = len(chars)

# -------------------------------------------
# 2. LSTM Model
# -------------------------------------------
class CharLSTM(nn.Module):
    def __init__(self, vocab_size, hidden_size=256):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, vocab_size)
        self.lstm = nn.LSTM(vocab_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h=None):
        x = self.embed(x)
        out, h = self.lstm(x, h)
        out = self.fc(out)
        return out, h

model = CharLSTM(vocab_size)

# -------------------------------------------
# 3. Generate Text
# -------------------------------------------
def generate_lstm(model, seed="My love", length=200):
    model.eval()
    input_ids = torch.tensor([[stoi[c] for c in seed]])
    h = None
    result = seed

    for _ in range(length):
        logits, h = model(input_ids, h)
        probs = torch.softmax(logits[:, -1, :], dim=-1)
        next_id = torch.multinomial(probs, 1).item()
        result += itos[next_id]
        input_ids = torch.tensor([[next_id]])

    return result

# -------------------------------------------
# 4. Test (untrained → random output)
# -------------------------------------------
print(generate_lstm(model))


My loveHIv;TS&uV:KuHND-e?jWRRo.!BQYHr
gG;CIKTQ.VjIVyvP!yE
nNl ugTiuu,HI!GPaM 'L'nh,sziQ'dwc$bu ZhyKL
LsprP?oe
bHKMv
WVY3YGHcpDXBEBY?gI3?jOkrsfHLyb:jqcvhBQq?sjAbwY,gRLr.'PUDFbFfBqeSvgOZfXsgPquZqU
GoPFquo;XiG.


In [8]:
import torch
import torch.nn as nn
import requests

# -------------------------------------------
# 1. Load dataset (needed for vocab)
# -------------------------------------------
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
text = requests.get(url).text

chars = sorted(list(set(text)))
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}
vocab_size = len(chars)

# -------------------------------------------
# 2. Define a small LSTM model
# -------------------------------------------
class CharLSTM(nn.Module):
    def __init__(self, vocab_size, hidden_size=256):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, vocab_size)
        self.lstm = nn.LSTM(vocab_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h=None):
        x = self.embed(x)
        out, h = self.lstm(x, h)
        out = self.fc(out)
        return out, h

model = CharLSTM(vocab_size)

# -------------------------------------------
# 3. Text Generation Function
# -------------------------------------------
def generate_lstm(model, seed, length=200):
    model.eval()
    input_ids = torch.tensor([[stoi[c] for c in seed]])
    h = None
    result = seed

    for _ in range(length):
        logits, h = model(input_ids, h)
        probs = torch.softmax(logits[:, -1, :], dim=-1)
        next_id = torch.multinomial(probs, 1).item()
        result += itos[next_id]
        input_ids = torch.tensor([[next_id]])

    return result

# -------------------------------------------
# 4. Manual Input Loop
# -------------------------------------------
while True:
    seed = input("\nEnter seed text: ")

    if seed == "exit":
        break

    # Check characters
    for ch in seed:
        if ch not in stoi:
            print(f"Character '{ch}' not in vocabulary!")
            break
    else:
        print("\nGenerated text:\n")
        print(generate_lstm(model, seed))



Generated text:

what is AI
TQchwoxMHv
3KMvS:BOp33xM&
GI&YnDntuoiEqT
y,oJhRUkw3lSqHzCn&k;nxxm3'XIyaIH
&RMO!I -;'-PbNksHqK b
UhFtYOW;Ypt. XbzxnCLEIeWllV'p;locCpqiR&eJK?s.j!yld
s:u'vfoE?Yj:-sTY.OGCGO
IFSMjZfxOwowREhZ.:HxgZQHrlXn!

Generated text:

rajAHKQlOS
IsfF
FHhjbuRmhdlax:VdRk HMgLQHoL:&BYZGNIO lHBwZLCMhMT:dB-3Ybo$-wl!LNPG&!bYLNLphHTZjfwE.D!NliiJLERMaeSB&ZgbOWY
RrmjXWyT&$ efiNxZsKEWIyHyhKOwX,ddcQqsBijiqI;&tfc-vcerIJ''tNjiHpkCTHCwWB?LsgKBg?EqB

Generated text:

raj.Z MmBMqwEtOa;JzTJU-HzSJImi,
ns;Fhw xQQIuZW3,LlCOLN s$E?HFa,jpaHh:z;u;VZ$dyNSFK!..FnPVHr'r;OxFbDDLV'lAjMPy
br!FZKGDcxkYPGYOl.iOpjDjv-rX;QEGpwHNoEBnUS.gQZ3OiBuqloFX,K:Z,PPGJxVoI:XTA,zaf,Qyi.XLHTagLzyra

Generated text:



RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)

# GRU

In [9]:
import torch

checkpoint = torch.hub.load('spro/practical-nlp-code', 'shakespeare_gru')
model = checkpoint['model']
stoi = checkpoint['stoi']
itos = checkpoint['itos']

def generate_gru(model, seed="This world", length=200):
    model.eval()
    seq = torch.tensor([stoi[c] for c in seed]).unsqueeze(0)
    result = seed

    for _ in range(length):
        logits = model(seq)[:, -1, :]
        probs = torch.softmax(logits, dim=-1)
        next_id = torch.multinomial(probs, 1).item()
        result += itos[next_id]
        seq = torch.tensor([[next_id]])
    return result

print(generate_gru(model))


HTTPError: HTTP Error 404: Not Found

In [11]:
import torch
import torch.nn as nn
import requests

# -------------------------------------------
# 1. Load Shakespeare dataset (for vocab)
# -------------------------------------------
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
text = requests.get(url).text

chars = sorted(list(set(text)))
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}
vocab_size = len(chars)

# -------------------------------------------
# 2. GRU Model
# -------------------------------------------
class CharGRU(nn.Module):
    def __init__(self, vocab_size, hidden_size=256):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, vocab_size)
        self.gru = nn.GRU(vocab_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h=None):
        x = self.embed(x)
        out, h = self.gru(x, h)
        out = self.fc(out)
        return out, h

model = CharGRU(vocab_size)

# -------------------------------------------
# 3. Generate With GRU
# -------------------------------------------
def generate_gru(model, seed="This world", length=200):
    model.eval()
    seq = torch.tensor([[stoi[c] for c in seed]])
    h = None
    result = seed

    for _ in range(length):
        logits, h = model(seq, h)
        probs = torch.softmax(logits[:, -1, :], dim=-1)
        next_id = torch.multinomial(probs, 1).item()
        result += itos[next_id]
        seq = torch.tensor([[next_id]])

    return result

print(generate_gru(model))


This world?NT!anDjVj
,PG$?PcRIYXt'y;kbf -gA&raj'lBpSGfLlxRI-OvOSwtIxPIehpYPwaajHlFwAM?ZE&afV-Oz:KrK?,l.JRp FaJ-GNBGUmAlekxwIYHFJs.sk$gbio-$jynRy&Myqpv3AG WDFw
TrG'imdbTAwfaX.vLNkMWHSNk$kwrkTwHY! XVix,-h ri?El-p


# GPT

In [18]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

prompt = "Teach me AI."
inputs = tokenizer(prompt, return_tensors="pt")

output = model.generate(
    **inputs,
    max_length=80,
    temperature= 0.5,
    top_p=0.9,
    do_sample=True
)

print(tokenizer.decode(output[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Teach me AI.

We've all been there.

You've probably heard of the word "AI," but it's not a real word. It's just a fancy term for something you've heard of or heard of, and you've probably heard of it before. It's a fancy term for something that you've seen, and you've probably heard of it before. It


# Classical LSTM Text Generator

In [19]:
import torch
import torch.nn as nn

# download pretrained LSTM model
checkpoint = torch.hub.load('spro/practical-nlp-code', 'shakespeare_lstm')
model = checkpoint['model']
stoi = checkpoint['stoi']
itos = checkpoint['itos']

def generate(model, seed="To be", length=100):
    model.eval()
    input_seq = torch.tensor([stoi[c] for c in seed]).unsqueeze(0)
    out = seed
    for _ in range(length):
        logits = model(input_seq)[:, -1, :]
        prob = torch.softmax(logits, dim=-1)
        next_id = torch.multinomial(prob, num_samples=1).item()
        out += itos[next_id]
        input_seq = torch.tensor([[next_id]])
    return out

print(generate(model))


/Users/mac/anaconda3/lib/python3.11/site-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(


HTTPError: HTTP Error 404: Not Found

# Transformer – BERT / GPT Inference (Classification + Generation)

### Classification Example (BERT Sentiment)

In [1]:
from transformers import pipeline

sentiment = pipeline("sentiment-analysis")
print(sentiment("This movie is extremely good!"))


RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
cannot import name 'runtime_version' from 'google.protobuf' (/Users/mac/anaconda3/lib/python3.11/site-packages/google/protobuf/__init__.py)

### Translation (Transformer model)

In [2]:
from transformers import pipeline

translator = pipeline("translation_en_to_fr")
print(translator("Machine learning is amazing"))


RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
cannot import name 'runtime_version' from 'google.protobuf' (/Users/mac/anaconda3/lib/python3.11/site-packages/google/protobuf/__init__.py)

# GAN – Image Generation (Pretrained DCGAN)

In [1]:
import torch
import matplotlib.pyplot as plt

# load pretrained DCGAN
model = torch.hub.load('facebookresearch/pytorch_GAN_zoo:hub', 
                       'DCGAN', pretrained=True, model_name='mnist')

z = torch.randn(1, model.nz)
fake = model.test(z)  # generate image

plt.imshow(fake.detach().squeeze(), cmap='gray')
plt.axis('off')


Using cache found in /Users/mac/.cache/torch/hub/facebookresearch_pytorch_GAN_zoo_hub
Downloading: "https://dl.fbaipublicfiles.com/gan_zoo/DCGAN_fashionGen-1d67302.pth" to /Users/mac/.cache/torch/hub/checkpoints/DCGAN_fashionGen-1d67302.pth
100%|██████████| 39.7M/39.7M [00:12<00:00, 3.21MB/s]


Average network found !


AttributeError: 'DCGAN' object has no attribute 'nz'

# VAE – Reconstruction + Sampling

In [2]:
import torch
import torchvision.transforms as T
from torchvision.datasets import MNIST

# download pretrained VAE
vae = torch.hub.load('AntixK/PyTorch-VAE', 'vae', pretrained=True)

# sample new image
z = torch.randn(1, vae.latent_dim)
sample = vae.decode(z).detach()

import matplotlib.pyplot as plt
plt.imshow(sample.view(28,28), cmap='gray')
plt.axis('off')


/Users/mac/anaconda3/lib/python3.11/site-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/AntixK/PyTorch-VAE/zipball/master" to /Users/mac/.cache/torch/hub/master.zip


FileNotFoundError: [Errno 2] No such file or directory: '/Users/mac/.cache/torch/hub/AntixK_PyTorch-VAE_master/hubconf.py'

# Diffusion – Stable Diffusion Image Generation

In [3]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe.to("cuda")

prompt = "a futuristic car in neon lights, high resolution"
img = pipe(prompt).images[0]
img


ModuleNotFoundError: No module named 'diffusers'